In [5]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import tensorflow_hub as hub

import os
from tensorflow.keras.callbacks import ModelCheckpoint

In [6]:
data_dir_training = "/Users/aaryangulia/Downloads/archive/asl_alphabet_train/asl_alphabet_train"

In [7]:
# Define the number of classes
num_classes = len(os.listdir(data_dir_training)) - 1

# Define the image size and batch size
img_size = (224, 224)
batch_size = 32

# Use ImageDataGenerator for data loading and augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    validation_split=0.2,  # Split data into training and validation sets
    rotation_range=20,  # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by up to 20% of the width
    height_shift_range=0.2,  # Randomly shift images vertically by up to 20% of the height
    shear_range=0.2,  # Shear intensity (shear angle in radians)
    zoom_range=0.2,  # Randomly zoom images by up to 20%
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Strategy for filling in newly created pixels
)

In [10]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam

In [11]:
train_generator = datagen.flow_from_directory(
    data_dir_training,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # For multi-class classification
    subset='training'  # Specify this as training data
)

# Create validation data generator
validation_generator = datagen.flow_from_directory(
    data_dir_training,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Specify this as validation data
)

Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


In [16]:
# getting pretrained model for transfer learning and defining model
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
download_model = hub.KerasLayer(url,input_shape=(224,224,3))
model = Sequential([
     download_model,
     Dense(29),
     Activation("softmax")
])
# compiling model
model.compile(optimizer=Adam(1e-3),
loss="categorical_crossentropy",
metrics=['accuracy'])

In [18]:
checkpoint_callback = ModelCheckpoint(filepath='epoch_custom_model.h5', 
                                      save_best_only=False, 
                                      verbose=1)
#training model
print("\n Model summary: ")
print(model.summary())
print("\n Model Training: ")
history = model.fit(
    train_generator,
    epochs=5,  # You can adjust the number of epochs based on your training needs
    validation_data=validation_generator,
    callbacks=[checkpoint_callback]
)


 Model summary: 
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 1001)              3540265   
                                                                 
 dense_2 (Dense)             (None, 29)                29058     
                                                                 
 activation_2 (Activation)   (None, 29)                0         
                                                                 
Total params: 3569323 (13.62 MB)
Trainable params: 29058 (113.51 KB)
Non-trainable params: 3540265 (13.51 MB)
_________________________________________________________________
None

 Model Training: 
Epoch 1/5
2175/2175 [==============================] - ETA: 0s - loss: 0.4263 - accuracy: 0.8680
Epoch 1: saving model to epoch_custom_model.h5
2175/2175 [==============================] - 1774s 815ms/step - loss: 0.4263 - accuracy:

/opt/anaconda3/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2175/2175 [==============================] - ETA: 0s - loss: 0.3085 - accuracy: 0.9015
Epoch 2: saving model to epoch_custom_model.h5
2175/2175 [==============================] - 1749s 804ms/step - loss: 0.3085 - accuracy: 0.9015 - val_loss: 0.8883 - val_accuracy: 0.7543
Epoch 3/5
2175/2175 [==============================] - ETA: 0s - loss: 0.2714 - accuracy: 0.9143
Epoch 3: saving model to epoch_custom_model.h5
2175/2175 [==============================] - 1612s 741ms/step - loss: 0.2714 - accuracy: 0.9143 - val_loss: 0.9468 - val_accuracy: 0.7482
Epoch 4/5
2175/2175 [==============================] - ETA: 0s - loss: 0.2617 - accuracy: 0.9174
Epoch 4: saving model to epoch_custom_model.h5
2175/2175 [==============================] - 1642s 755ms/step - loss: 0.2617 - accuracy: 0.9174 - val_loss: 0.9188 - val_accuracy: 0.7604
Epoch 5/5
2175/2175 [==============================] - ETA: 0s - loss: 0.2546 - accuracy: 0.9202
Epoch 5: saving model to epoch_custom_model.h5
2175/2175 [=========

In [ ]:
# evaluating model
print("\n Model Evaluation: ")
model.evaluate(validation_generator)

In [96]:
from keras.preprocessing import image

#load the model we saved
test_image = image.load_img('/Users/aaryangulia/Downloads/archive-2/Test_Alphabet/A/1e5fcac0-0494-4fcb-9dcc-a7c17403abe9.rgb_0000.png', target_size=(224, 224,3))


In [98]:
img_array = tf.keras.utils.img_to_array(test_image)/255.0
img_array = tf.expand_dims(img_array, 0)

In [99]:
predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 [==============================] - 0s 49ms/step
This image most likely belongs to A with a 8.82 percent confidence.


In [2]:
# Load the existing model
model = tf.keras.models.load_model("models/english_model.h5", custom_objects={'KerasLayer': hub.KerasLayer})

# Register the KerasLayer
tf.keras.utils.get_custom_objects()['KerasLayer'] = hub.KerasLayer

# Save the model to the TensorFlow.js format
tf.saved_model.save(model, "models/saved_model")

INFO:tensorflow:Assets written to: models/saved_model/assets


INFO:tensorflow:Assets written to: models/saved_model/assets
